In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.style.use('default')
# plt.style.use('dark_background')
%matplotlib inline

In [2]:
whiskey = pd.read_csv('./Scraping/Scraped_Data/master_whiskey.csv', encoding='latin1')

# drop f's and no grades
whiskey = whiskey[(whiskey['grade'] != 'f') & (whiskey['grade'] != 'No Grade')].copy()

# fill missing ages, vintages, region, abv
whiskey['age'].fillna('No Age', inplace=True)
whiskey['vint'].fillna('No Vint', inplace=True)
whiskey['region'].fillna('No Region', inplace=True)
whiskey['abv'].fillna('No Abv', inplace=True)

# drop "yrs" for the age
whiskey['age'] = whiskey['age'].map(lambda x : x.split(' yrs')[0])

# drop missing revies
whiskey = whiskey.dropna().copy()

In [3]:
beer = pd.read_csv('./Scraping/Scraped_Data/master_beer.csv', encoding='latin1')

# remove reviews without text reviews
beer = beer[beer['r_text'].str.contains('No Review') == False]
beer = beer[beer['breakdown'].str.contains('look:') == True]

# fill missing user name
beer['username'].fillna('Missing Username', inplace=True)
# fill nan ibu's
beer['ibu'].fillna('No IBU', inplace=True)

In [5]:
beer.head()

,name,url,score_x,ratings,brewery,style,abv,ibu,brewer_notes,score_y,breakdown,username,r_text
0,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.82,look: 4.25 | smell: 4.75 | taste: 5 | feel: 4....,Nobel96,No Review
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.79,look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 ...,Batman11,No Review
2,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.00,look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...,Bloodsnot77,No Review
3,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.14,look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 ...,jrc1093,No Review
4,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.09,look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 ...,Maudrey,No Review


In [10]:
beer.shape

(630889, 13)

In [15]:
beer = beer[beer['r_text'].str.contains('No Review') == False]

In [16]:
beer.shape

(367279, 13)

In [20]:
beer[beer['breakdown'].str.contains('look:') != True]['name'].value_counts()

Raison D'Être                                    2175
OAKED Arrogant Bastard Ale                       2119
Samuel Adams Latitude 48 IPA                     2065
A Little Sumpin' Wild                            2031
Sublimely Self-Righteous Black IPA               1979
Red's Rye IPA                                    1957
Samuel Adams Cherry Wheat                        1914
Arctic Panzer Wolf                               1896
Samuel Adams Noble Pils                          1883
Stone Smoked Porter                              1877
Hop Head Red Ale                                 1864
Sierra Nevada Ruthless Rye IPA                   1842
Rogue Chocolate Stout                            1835
Northern Hemisphere Harvest Wet Hop IPA          1829
ApriHop                                          1812
Miles Davis' Bitches Brew                        1810
75 Minute IPA                                    1807
Sierra Nevada Porter                             1792
Hop Rod Rye                 

In [23]:
beer = beer[beer['breakdown'].str.contains('look:') == True]

In [24]:
beer.shape

(62487, 13)

In [25]:
beer.isnull().sum()

name              0
url               0
score_x           0
ratings           0
brewery           0
style             0
abv               0
ibu             470
brewer_notes      0
score_y           0
breakdown         0
username          1
r_text            0
dtype: int64

In [30]:
beer['username'].fillna('Missing Username', inplace=True)

In [36]:
beer['ibu'].value_counts()

No IBU    47649
60         2112
50         1674
70         1434
100         798
75          568
69          558
6           530
37          515
38          477
98          396
90          377
45          327
120         322
55          310
93          296
65          271
68          219
82          216
95          214
80          214
33          208
102         205
44          204
12          203
8           198
85          194
20          193
35          193
18          182
42          174
51          161
28          159
86          136
25          130
Name: ibu, dtype: int64

In [39]:
beer['ibu'].fillna('No IBU', inplace=True)

In [40]:
beer.isnull().sum()

name            0
url             0
score_x         0
ratings         0
brewery         0
style           0
abv             0
ibu             0
brewer_notes    0
score_y         0
breakdown       0
username        0
r_text          0
dtype: int64

In [41]:
whiskey.isnull().sum()

username      0
grade         0
review        6
w_name        0
bottler       0
age         125
w_type        0
vint        213
subtype       0
abv           2
region       47
price         0
avaib         0
url           0
dtype: int64

In [42]:
whiskey['age'].value_counts()

21 yrs     22
NAS yrs    21
16 yrs     18
8 yrs      17
19 yrs     17
18 yrs     17
12 yrs     16
7 yrs      14
14 yrs     14
25 yrs     12
15 yrs     12
10 yrs     11
5 yrs       9
23 yrs      9
20 yrs      8
26 yrs      8
17 yrs      7
2 yrs       6
13 yrs      6
24 yrs      6
30 yrs      6
3 yrs       5
36 yrs      5
29 yrs      5
33 yrs      5
50 yrs      5
11 yrs      4
27 yrs      3
37 yrs      3
40 yrs      2
31 yrs      2
22 yrs      2
35 yrs      2
9 yrs       1
6 yrs       1
4 yrs       1
32 yrs      1
Name: age, dtype: int64

In [45]:
whiskey[whiskey['age'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
2,Andy,bplus,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,NaN,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
3,Fuji,bminus,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,NaN,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
4,Fuji,b,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,NaN,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
5,Andy,b,n: Cherry Kirsch in a metal thermos. p: Dipp...,Aberlour A'bunadh Batch 47,OB,NaN,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
6,Andy,b,n: Plum pie whipped cream p: Hot and Christm...,Aberlour A'bunadh Batch 50,OB,NaN,Scotch,NaN,Single Malt,59.6,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...


In [46]:
whiskey['age'].fillna('No Age', inplace=True)

In [49]:
whiskey[whiskey['vint'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
0,Ries,b,n: fuller rich nose; full vanilla caramels van...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12 yrs,Scotch,NaN,Single Malt,48.0,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
1,Andy,bminus,n: Gym socks across the room p: Wierd. Grea...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12 yrs,Scotch,NaN,Single Malt,48.0,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
2,Andy,bplus,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
3,Fuji,bminus,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
4,Fuji,b,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,No Age,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...


In [50]:
whiskey['vint'].fillna('No Vint', inplace=True)

In [51]:
whiskey[whiskey['region'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
8,Andy,b,n: Toasted oats and rose petals p: Cheerios....,Amrut Rye,OB,5 yrs,World,No Vint,Indian,50.0,NaN,180,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...
9,Sku,cminus,Nose: Grainy with vanilla notes.Palate: Raw an...,Amrut Rye,OB,5 yrs,World,No Vint,Indian,50.0,NaN,180,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...
48,Andy,cplus,n: Cheerios and cardboard. p: Neutral and bit...,Black Bull 21,Duncan Taylor,21 yrs,Scotch,No Vint,Blend,50.0,NaN,150,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
49,Fuji,cplus,Semi sweet vanilla with fruit (apple and pear),Black Bull 21,Duncan Taylor,21 yrs,Scotch,No Vint,Blend,50.0,NaN,150,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
74,Andy,cplus,n: A bit sour. Like a bad finish. p: Wood va...,Cadenhead Creations 20yr Batch 1,Cadenhead,20 yrs,Scotch,No Vint,Blend,46.0,NaN,99,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...


In [52]:
whiskey['region'].fillna('No Region', inplace=True)

In [63]:
whiskey['age'] = whiskey['age'].map(lambda x : x.split(' yrs')[0])

In [60]:
for i in whiskey['age']:
    i = i.split(' yrs')[0]
    print(i)

12
12
No Age
No Age
No Age
No Age
No Age
No Age
5
5
16
16
No Age
21
21
No Age
NAS
NAS
No Age
No Age
No Age
14
17
15
No Age
No Age
No Age
No Age
No Age
11
25
15
15
15
No Age
No Age
9
18
19
17
17
32
19
19
19
19
35
15
21
21
15
26
26
26
26
20
No Age
12
12
12
12
11
11
No Age
No Age
18
18
23
23
23
No Age
40
40
23
20
20
No Age
27
30
30
30
31
31
21
21
15
14
20
8
5
5
5
5
12
12
12
12
No Age
No Age
No Age
No Age
No Age
No Age
3
3
3
17
17
7
2
No Age
No Age
No Age
22
22
No Age
No Age
No Age
No Age
No Age
No Age
No Age
No Age
No Age
7
7
7
7
23
13
13
18
18
18
18
No Age
No Age
No Age
No Age
No Age
20
20
No Age
13
13
10
8
12
No Age
No Age
No Age
No Age
No Age
No Age
24
24
24
24
14
14
19
8
8
21
21
14
No Age
No Age
No Age
No Age
14
15
15
21
19
19
8
8
8
No Age
NAS
NAS
NAS
21
No Age
No Age
No Age
No Age
No Age
No Age
37
No Age
No Age
No Age
33
33
33
33
33
19
19
23
14
14
18
No Age
No Age
No Age
No Age
7
No Age
No Age
No Age
10
No Age
No Age
25
25
14
8
8
24
24
20
7
No Age
12
No Age
No Age
No Age
No Age
No Ag

In [64]:
whiskey['age'].value_counts()

No Age    125
21         22
NAS        21
16         18
8          17
18         17
19         17
12         16
14         14
7          14
25         12
15         12
10         11
23          9
5           9
20          8
26          8
17          7
13          6
2           6
24          6
30          6
29          5
50          5
33          5
3           5
36          5
11          4
27          3
37          3
40          2
22          2
31          2
35          2
32          1
4           1
9           1
6           1
Name: age, dtype: int64

In [85]:
whiskey.isnull().sum()

username    0
grade       0
review      6
w_name      0
bottler     0
age         0
w_type      0
vint        0
subtype     0
abv         0
region      0
price       0
avaib       0
url         0
dtype: int64

In [93]:
whiskey2.isnull().sum()

username    0
grade       0
review      0
w_name      0
bottler     0
age         0
w_type      0
vint        0
subtype     0
abv         0
region      0
price       0
avaib       0
url         0
dtype: int64

In [92]:
whiskey2 = whiskey.dropna().copy()

In [91]:
whiskey.shape

(428, 14)

In [86]:
whiskey['review'].dropna(inplace=True)

In [73]:
whiskey.shape

(428, 14)

## Modeling

In [ ]:
stop = stopwords.words('english')